***
## <center> EXTRACCIÓN DE DATOS DESDE UN FORMATO RASTER </center>
### <center>CÓDIGO      :     PYTHON </center>
### <center>LIBRERIAS   :     RASTERIO </center>
### <center>POR         :     VICTOR B. VELÁSQUEZ HUAMÁN </center>
### <center>NOMBRE      :     PYFRED </center>
### <center>FECHA       :     MARCH 23, 2019</center>
***
>TEMA: Extracción de datos de Precipitación desde un Formato Raster del Producto  Grillado **PISCO-SENAMHI** , **GPM** , **TRMM**, **CHIRPS**

***Intalación de Librerias***

In [20]:
#!pip install xlrd==1.2.0 

***Importación de Librerias***

In [21]:
from tqdm.notebook import tnrange, tqdm_notebook
import numpy as np
import rasterio
import xlrd
import xlwt
import os

### 1.0  MODELOS LÓGICOS
<p style="text-align: justify;"> El modelo logico hace referencia a como se muestrean y organizan las variables y objetos para lograr una representación lo más adecuada posible. En un SIG existen básicamente dos modelos lógicos que se conocen como <b>formato raster y formato vectorial</b> y que dan lugar a los dos grandes tipos de capas de información espacial.(MappingGIS,<a href="https://mappinggis.com/2015/12/los-formatos-gis-raster-mas-populares/">Visit MappingGIS</a>) </p>


#### 1.1  MODELOS RASTER
<p style="text-align: justify;"> Es una imagen dicretizada de manera regular las que se denominan (PIXELS), las cuales continen  valores en cada una de ellas, estos dependen del tamaño de resolucion que se les atribuye, asi mismo cabe indidar que tambien pueden tener valores multiples en una unidad de Pixel, a estos se denominana bandas (RATER MULTIBANDA). Los datos de del producto grillado PISCO-SENAMHI, son Raster que una sola banda, a una resolucion de 0.1° equivalente a 10 km, las cauales contienen valores de precipitacion diaria.  </p>
<figure>
<img src = "./Imagenes/Raster_dataset.PNG" alt = "Sol" style="width:400px"> </img>
<figcaption> (Fuente :Adaptado desde,<a href="https://docs.qgis.org/2.14/es/docs/gentle_gis_introduction/raster_data.html">Documentación para Qgis 2.14</a>)
</figcaption> 
</figure>



#### 1.2  MODELOS VECTORIAL
<p style="text-align: justify;">Un SIG vectorial se define por la representación vectorial de sus datos geográficos. De acuerdo a las peculiaridades de este modelo de datos, los objetos geográficos se representan explícitamente y, junto a sus características espaciales, se asocian sus valores temáticos.(<a href="https://www.geogra.uah.es/gisweb/1modulosespanyol/IntroduccionSIG/GISModule/GIST_Vector.htm#:~:text=Un%20SIG%20vectorial%20se%20define,se%20asocian%20sus%20valores%20tem%C3%A1ticos.">Gis self Learning tool</a>)</p>




### 2.0  EJEMPLO APLICATIVO
<p style="text-align: justify;"> Para la Sub Cuenca del río Huarpa, obtener los datos de precipitación del prodcuto grillado PISCO-SENAMHi, de forma distibuida y en los centros de gravedad de las microcuencas</p>

<figure>
<img src = "./Imagenes/SubCuenca_Angaraes.PNG" alt = "Sol" style="width:800px"> </img>
<figcaption> (Fuente :Elaboracion Propia,<a href="https://drive.google.com/drive/folders/1SxBFJkKzgfn1KRmalPmrqFrsAGAVHcjp?usp=sharing">Qgis</a>)
</figcaption> 
</figure>


|Código|Xgeo| YGeo   | Nombre |
|---------|--|------|------|
|499621|-74.3926|-12.933|Estacion Sintetica 01|
|499622|-74.5243|-13.0552|Estacion Sintetica 02|
|499623|-74.5666|-12.8864|Estacion Sintetica 03|
|499624|-74.7334|-12.7827|Estacion Sintetica 04|
|499625|-74.7108|-12.9252|Estacion Sintetica 05|
|499626|-74.693|-13.1627|Estacion Sintetica 06|
|499627|-74.8197|-13.0535|Estacion Sintetica 07|
|499628|-74.9634|-12.9013|Estacion Sintetica 08|
|499629|-74.9634|-13.044|Estacion Sintetica 09|


In [22]:
path = "F:\\Precipitacion_Remota\\Download_Pisco"

In [25]:
class Extract_Pisco():   
    def __init__(self):
        xls = xlrd.open_workbook(path +"\\Point.xlsx")
        libro = xls.sheet_by_index(7)
        self.Name_Estaciones = libro.col_values(0)[1::]
        self.x = libro.col_values(1)[1::]
        self.y = libro.col_values(2)[1::]
        
    def DataPre(self):
        os.chdir(path) 
        fileDir= path
        fileExt = r".tif"
        Lis = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
        Lis.sort()
        Inicio = '1981-01-01.tif'
        Fin    = '2018-07-31.tif'
        Rini = Lis.index(Inicio)
        Rend = Lis.index(Fin)
        self.Lis = Lis[Rini:Rend+1]
        self.ListPre = []
        for i in tnrange (0,len(self.Lis),desc = "!! Extraendo Datos !!"):
            with rasterio.open(self.Lis[i]) as Data:
                Band = Data.read(1)
                X,Y  = Data.index(self.x,self.y)
                Pre  = Band[X,Y]
                self.ListPre.append(Pre)
                
    def ExportExcel(self):
        style0 = xlwt.easyxf('font: name Century Gothic, color-index blue, bold on, height 180 ;align: vert centre, horz center')
        style1 = xlwt.easyxf('font: name Century Gothic, color-index black, bold off, height 180;align: vert centre, horz center', num_format_str='#,##0.000')
        style2 = xlwt.easyxf('font: name Century Gothic, color-index black, bold on, height 180;align: vert centre, horz center', num_format_str='#,##0.000')
        wb = xlwt.Workbook()
        ws = wb.add_sheet('Export')
        ws.write(0, 0, u'Fecha', style0)

        for i in range (0,len(self.Name_Estaciones)):
            ws.write(0, i+1, self.Name_Estaciones[i], style0)

        for i in range(0, len(self.Lis)):
            ws.write(i+1, 0, self.Lis[i][:-4], style2) 

            for j in range (0,len(self.Name_Estaciones)):
                if (len(self.Name_Estaciones) ==1):
                    ws.write(i+1, j+1, self.ListPre[i].astype(np.float64),style1)
                else:
                    ws.write(i+1, j+1, self.ListPre[i][j].astype(np.float64),style1)
        wb.save('Reporte_Fred.xls')

In [26]:
Pisco = Extract_Pisco()
Pisco.DataPre()
Pisco.ExportExcel()


!! Extraendo Datos !!:   0%|          | 0/13726 [00:00<?, ?it/s]

### 3.0 REFERENCIAS
<ul>
 <li> Senamhi </li>
    
[SENAMHI](https://www.senamhi.gob.pe/?p=observacion-de-inundaciones).
   
  <li> Libreria Rasterio </li>
    
[Rasterio](https://rasterio.readthedocs.io/en/latest/installation.html).   
    
  <li> Libreria Xlrd</li>
    
[Xlrd](https://pypi.org/project/xlrd/).           
    
<li> GitHub Victor Brian Velásquez Huamán </li>
    
[GitHub](https://github.com/bryan142857/Python).
    
</ul>
